samarasinghe et.al

## CNN architecture [kim yoon (2011)]

The CNN had 4 convolutional layers with each having 256
convolutional filters with window sizes of 1, 2, 3 and 4 with the activation function as relu. These convolutional filters are able to extract features with 1 to 4 words. Before passing data into the CNN, padding of 0s had to be applied due to the different lengths of the sentences. Each convolutional layer was followed by a pooling layer that uses max pooling as its operator. Subsequently, a dropout layer was applied in order to avoid over fitting. After the convolutional layers, outputs of all dropout layers were concatenated and flattened to be passed into a fully connected neural network. Then, 2 layers with 16 and 1 neurons were used in the fully connected neural network with relu and sigmoid as activation functions, respectively.
E.

In [ ]:
import neptune
from neptunecontrib.monitoring.metrics import expand_prediction, log_class_metrics, log_binary_classification_metrics, log_classification_report,log_confusion_matrix,log_prediction_distribution
import os
from dotenv import load_dotenv

load_dotenv()
NEPTUNE_PROJECT= os.getenv('NEPTUNE_PROJECT')
NEPTUNE_API_TOKEN = os.getenv(('NEPTUNE_API_TOKEN'))
neptune.init(project_qualified_name= NEPTUNE_PROJECT,api_token=NEPTUNE_API_TOKEN) 
             

In [3]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import sys
sys.path.insert(1,'G:\\Github\\Sinhala-Hate-Speech-Detection')
import utills

In [4]:
# load datasets
path = '../Datasets/processed/no_preprocessing/'
df_A = pd.read_csv(path+'df_A.csv') 
# df_B = pd.read_csv(path+'df_B.csv')    # fb dataset -kaggle
# df_C = pd.read_csv(path+'df_C.csv') 
# df_D = pd.read_csv(path+'df_D.csv')  
# df_all = pd.read_csv(path+'df_all.csv') 

In [5]:
# Padding the data samples to a maximum review length in words
max_words = 450
X_train,X_test,Y_train,Y_test = utills.prepare_dataset(df_A,"df_A")
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)


df_A 6468
X train (4527,) Y train (4527,) X test (1941,) Y test (1941,)


ValueError: invalid literal for int() with base 10: 'මාත් මේ පුක වලලං මැරෙන්න හදනගමං '

In [ ]:

# Building the CNN Model
model = Sequential()      # initilaizing the Sequential nature for CNN model

# Adding the embedding layer which will take in maximum of 450 words as input and provide a 32 dimensional output of those words which belong in the top_words dictionary
model.add(Embedding(top_words, 32, input_length=max_words))


model.add(Conv1D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
# Fitting the data onto model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=128, verbose=2)

# Getting score metrics from our model
scores = model.evaluate(X_test, y_test, verbose=0)

# Displays the accuracy of correct sentiment prediction over test data
print("Accuracy: %.2f%%" % (scores[1]*100))


In [ ]:
cnn = Sequential()
cnn.add(Embedding(input_dim=137, 
                    output_dim=100, 
                    input_length=1223))
cnn.add(Conv1D(filters=100, 
                 kernel_size=5, 
                 activation='relu'))
cnn.add(Flatten())
cnn.add(Dense(100, activation='relu'))
cnn.add(Dense(1, activation='sigmoid'))cnn.compile(optimizer='adam', 
              loss="binary_crossentropy",
              metrics=['accuracy'])cnn.fit(X_train, y_train, 
          epochs=10, 
          verbose=1)

In [ ]:
train_score = 0.9626
test_score = cnn.evaluate(X_test, y_test)[1]
y_pred = cnn.predict_classes(X_test)
y_pred = y_pred.flatten()_f1_score = f1_score(y_test, y_pred)

In [ ]:
#load embeddings
print('loading word embeddings...')
embeddings_index = {}
f = codecs.open('../Embedding_models/cc.si.300.bin.gz', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index))

In [6]:
import fasttext
import fasttext.util
ft = fasttext.load_model('../Embedding_models/cc.si.300.bin')
ft.get_dimension()

ModuleNotFoundError: No module named 'fasttext'

In [ ]:
max_seq_len = np.round(train_df['doc_len'].mean() + train_df['doc_len'].std()).astype(int)
print("tokenizing input data...")
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(processed_docs_train + processed_docs_test)  #leaky
word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
word_seq_test = tokenizer.texts_to_sequences(processed_docs_test)
word_index = tokenizer.word_index
print("dictionary size: ", len(word_index))

#pad sequences
word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_test = sequence.pad_sequences(word_seq_test, maxlen=max_seq_len)

In [ ]:


#training params
batch_size = 256 
num_epochs = 8 

#model parameters
num_filters = 64 
embed_dim = 300 
weight_decay = 1e-4



In [ ]:
#embedding matrix
print('preparing embedding matrix...')
words_not_found = []
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_dim))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

In [ ]:
print("sample words not found: ", np.random.choice(words_not_found, 10))

In [ ]:


#CNN architecture
print("training CNN ...")
model = Sequential()
model.add(Embedding(nb_words, embed_dim,
          weights=[embedding_matrix], input_length=max_seq_len, trainable=False))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dense(num_classes, activation='sigmoid'))  #multi-label (k-hot encoding)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()



In [ ]:

#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]

In [ ]:

#model training
hist = model.fit(word_seq_train, y_train, batch_size=batch_size, epochs=num_epochs, callbacks=callbacks_list, validation_split=0.1, shuffle=True, verbose=2)

In [ ]:
y_test_pred = model.predict(word_seq_test)

In [ ]:
#generate plots
plt.figure()
plt.plot(hist.history['loss'], lw=2.0, color='b', label='train')
plt.plot(hist.history['val_loss'], lw=2.0, color='r', label='val')
plt.title('CNN sentiment')
plt.xlabel('Epochs')
plt.ylabel('Cross-Entropy Loss')
plt.legend(loc='upper right')
plt.show()

In [ ]:


plt.figure()
plt.plot(hist.history['acc'], lw=2.0, color='b', label='train')
plt.plot(hist.history['val_acc'], lw=2.0, color='r', label='val')
plt.title('CNN sentiment')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.show()



## word2vec

In [1]:
from gensim.models import word2vec
#model = Word2Vec.load_word2vec_format("G:/Github/Sinhala-Hate-Speech-Detection/Embedding_models/cbow_300.w2v")
path = "../Embedding_models/cbow_300.w2v"
model = word2vec.Word2Vec.load(path)
#gensim.models.KeyedVectors.load_word2vec_format
words = list(model.wv.vocab)
print(len(words))
print(words)
model.most_similar('නීතිය')

NameError: name 'word2vec' is not defined